### Problem Description
This problem is an extension of the Shipping Cost Case Study 2. In this problem, we have six end distributors, each with a known demand for a product. Distributor demand can be satisfied from a set of four depots, or directly from a set of two factories. Each depot can support a maximum volume of product moving through it, and each factory can produce a maximum amount of product. There are known costs associated with transporting the product, from a factory to a depot, from a depot to a distributor, or from a factory directly to a distributor. This extension provides the opportunity to choose which four of the six possible depots to open. It also provides an option of expanding capacity at one specific depot.

**Sets**
- Factories = {NewDelhi, Vishakhapatnam}
- Depots = {Ahmedabad, Kolkata, Coimbatore, Nagpur, Patna, Chennai}
- Distributors = {C1, C2, C3, C4, C5, C6}
- Cities = Factories ∪ Depots ∪ Distributors

**Parameters**
- cost: Cost of shipping one ton from source  s  to destination  t.
- supply: Maximum possible supply from factory  f  (in tons).
- throughput: Maximum possible flow through depot  d  (in tons).
- demand: Demand for goods at distributor  c  (in tons).
- opencost: Cost of opening depot  d  (in dollars).

**Decision Variables**
- flow: Quantity of goods (in tons) that is shipped from source  s  to destionation  t.
- open: Is depot d open?
- expand: Should Kolkata be expanded?

**Objective Function**
- Cost: Minimize total shipping costs. i.e, **Minimize = (cost ∗ flow) + (opencost * open) + (3000 * expand)**
 
**Constraints**

Factory output: Flow of goods from a factory must respect maximum capacity.
- flow ≤ supply
 
Distributor demand: Flow of goods must meet distributor demand.
- flow = demand
 
Depot flow: Flow into a depot equals flow out of the depot.
- flow = flow (from source to depot and depot to destination)
 
Depot capacity (without Kolkata): Flow into a depot must respect depot capacity.
- flow ≤ throughput * open

Depot capacity (for Kolkata): Flow into a depot must respect depot capacity.
- flow ≤ throughput + 20000 * expand

Open depots: At most 4 open depots (no choice for Kolkata or Coimbatore).
- open ≤ 4

In [1]:
from pulp import *
import pandas as pd

In [2]:
# Create dictionaries to capture factory supply limits, depot throughput limits, and distributor demand.
supply = dict({'NewDelhi': 150000,
               'Vishakhapatnam': 200000})

throughput = dict({'Ahmedabad': 70000,
                   'Kolkata': 50000,
                   'Coimbatore': 100000,
                   'Nagpur': 40000,
                   'Patna': 30000,
                   'Chennai': 25000})

opencost = dict({'Ahmedabad': 10000,
                 'Kolkata': 0,
                 'Coimbatore': 0,
                 'Nagpur': 5000,
                 'Patna': 12000,
                 'Chennai': 4000})

demand = dict({'C1': 50000,
               'C2': 10000,
               'C3': 40000,
               'C4': 35000,
               'C5': 60000,
               'C6': 20000})

In [3]:
# Create a dictionary to capture shipping costs.
cost = dict({'NewDelhi': {'Ahmedabad' : 0.5, 'Kolkata': 0.5, 'Coimbatore': 1.0, 'Nagpur': 0.2, 'Patna': 0.6,
                          'Chennai': 0.4, 'C1': 1.0, 'C3': 1.5, 'C4': 2.0, 'C6': 1.0},
             'Vishakhapatnam': {'Kolkata': 0.3, 'Coimbatore': 0.5, 'Nagpur': 0.2, 'Patna': 0.4,
                                'Chennai': 0.3, 'C1': 2.0},
             'Ahmedabad': {'C2': 1.5, 'C3': 0.5, 'C5': 1.5, 'C6': 1.0},
             'Kolkata': {'C1': 1.0, 'C2': 0.5, 'C3': 0.5, 'C4': 1.0, 'C5': 0.5},
             'Coimbatore': {'C2': 1.5, 'C3': 2.0, 'C5': 0.5, 'C6': 1.5},
             'Nagpur': {'C3': 0.2, 'C4': 1.5, 'C5': 0.5, 'C6': 1.5},
             'Patna': {'C1': 1.2, 'C2': 0.6, 'C3': 0.5, 'C5': 0.3, 'C6': 0.8},
             'Chennai': {'C2': 0.4, 'C4': 0.5, 'C5': 0.6, 'C6': 0.9}})
cost

{'NewDelhi': {'Ahmedabad': 0.5,
  'Kolkata': 0.5,
  'Coimbatore': 1.0,
  'Nagpur': 0.2,
  'Patna': 0.6,
  'Chennai': 0.4,
  'C1': 1.0,
  'C3': 1.5,
  'C4': 2.0,
  'C6': 1.0},
 'Vishakhapatnam': {'Kolkata': 0.3,
  'Coimbatore': 0.5,
  'Nagpur': 0.2,
  'Patna': 0.4,
  'Chennai': 0.3,
  'C1': 2.0},
 'Ahmedabad': {'C2': 1.5, 'C3': 0.5, 'C5': 1.5, 'C6': 1.0},
 'Kolkata': {'C1': 1.0, 'C2': 0.5, 'C3': 0.5, 'C4': 1.0, 'C5': 0.5},
 'Coimbatore': {'C2': 1.5, 'C3': 2.0, 'C5': 0.5, 'C6': 1.5},
 'Nagpur': {'C3': 0.2, 'C4': 1.5, 'C5': 0.5, 'C6': 1.5},
 'Patna': {'C1': 1.2, 'C2': 0.6, 'C3': 0.5, 'C5': 0.3, 'C6': 0.8},
 'Chennai': {'C2': 0.4, 'C4': 0.5, 'C5': 0.6, 'C6': 0.9}}

In [4]:
sources = []
destinations = []
for i in cost.keys():
    for j in cost[i]:
        sources.append(i)
        destinations.append(j)
possibleFlows = list(zip(sources, destinations))
possibleFlows

[('NewDelhi', 'Ahmedabad'),
 ('NewDelhi', 'Kolkata'),
 ('NewDelhi', 'Coimbatore'),
 ('NewDelhi', 'Nagpur'),
 ('NewDelhi', 'Patna'),
 ('NewDelhi', 'Chennai'),
 ('NewDelhi', 'C1'),
 ('NewDelhi', 'C3'),
 ('NewDelhi', 'C4'),
 ('NewDelhi', 'C6'),
 ('Vishakhapatnam', 'Kolkata'),
 ('Vishakhapatnam', 'Coimbatore'),
 ('Vishakhapatnam', 'Nagpur'),
 ('Vishakhapatnam', 'Patna'),
 ('Vishakhapatnam', 'Chennai'),
 ('Vishakhapatnam', 'C1'),
 ('Ahmedabad', 'C2'),
 ('Ahmedabad', 'C3'),
 ('Ahmedabad', 'C5'),
 ('Ahmedabad', 'C6'),
 ('Kolkata', 'C1'),
 ('Kolkata', 'C2'),
 ('Kolkata', 'C3'),
 ('Kolkata', 'C4'),
 ('Kolkata', 'C5'),
 ('Coimbatore', 'C2'),
 ('Coimbatore', 'C3'),
 ('Coimbatore', 'C5'),
 ('Coimbatore', 'C6'),
 ('Nagpur', 'C3'),
 ('Nagpur', 'C4'),
 ('Nagpur', 'C5'),
 ('Nagpur', 'C6'),
 ('Patna', 'C1'),
 ('Patna', 'C2'),
 ('Patna', 'C3'),
 ('Patna', 'C5'),
 ('Patna', 'C6'),
 ('Chennai', 'C2'),
 ('Chennai', 'C4'),
 ('Chennai', 'C5'),
 ('Chennai', 'C6')]

In [5]:
# Model Definition
model = LpProblem("Shipping_Cost_Optimization_2", LpMinimize)

In [6]:
plant, depot, distributor = supply.keys(), throughput.keys(), demand.keys()
plant, depot, distributor

(dict_keys(['NewDelhi', 'Vishakhapatnam']),
 dict_keys(['Ahmedabad', 'Kolkata', 'Coimbatore', 'Nagpur', 'Patna', 'Chennai']),
 dict_keys(['C1', 'C2', 'C3', 'C4', 'C5', 'C6']))

In [7]:
# Decision Variables
flowVar = LpVariable.dicts(name = 'Flow', 
                           indexs = [(src+'_'+dest) for src in cost.keys() for dest in cost[src]], 
                           lowBound = 0, cat = LpContinuous)
flowVar

{'NewDelhi_Ahmedabad': Flow_NewDelhi_Ahmedabad,
 'NewDelhi_Kolkata': Flow_NewDelhi_Kolkata,
 'NewDelhi_Coimbatore': Flow_NewDelhi_Coimbatore,
 'NewDelhi_Nagpur': Flow_NewDelhi_Nagpur,
 'NewDelhi_Patna': Flow_NewDelhi_Patna,
 'NewDelhi_Chennai': Flow_NewDelhi_Chennai,
 'NewDelhi_C1': Flow_NewDelhi_C1,
 'NewDelhi_C3': Flow_NewDelhi_C3,
 'NewDelhi_C4': Flow_NewDelhi_C4,
 'NewDelhi_C6': Flow_NewDelhi_C6,
 'Vishakhapatnam_Kolkata': Flow_Vishakhapatnam_Kolkata,
 'Vishakhapatnam_Coimbatore': Flow_Vishakhapatnam_Coimbatore,
 'Vishakhapatnam_Nagpur': Flow_Vishakhapatnam_Nagpur,
 'Vishakhapatnam_Patna': Flow_Vishakhapatnam_Patna,
 'Vishakhapatnam_Chennai': Flow_Vishakhapatnam_Chennai,
 'Vishakhapatnam_C1': Flow_Vishakhapatnam_C1,
 'Ahmedabad_C2': Flow_Ahmedabad_C2,
 'Ahmedabad_C3': Flow_Ahmedabad_C3,
 'Ahmedabad_C5': Flow_Ahmedabad_C5,
 'Ahmedabad_C6': Flow_Ahmedabad_C6,
 'Kolkata_C1': Flow_Kolkata_C1,
 'Kolkata_C2': Flow_Kolkata_C2,
 'Kolkata_C3': Flow_Kolkata_C3,
 'Kolkata_C4': Flow_Kolkata_C4

In [8]:
openDepot = LpVariable.dicts(name = 'Open', 
                             indexs = depot, 
                             lowBound = 0, cat = LpBinary)
openDepot

{'Ahmedabad': Open_Ahmedabad,
 'Kolkata': Open_Kolkata,
 'Coimbatore': Open_Coimbatore,
 'Nagpur': Open_Nagpur,
 'Patna': Open_Patna,
 'Chennai': Open_Chennai}

In [9]:
expandKolkata = LpVariable.dicts(name = 'Expand', 
                                 indexs = [d for d in depot if d == 'Kolkata'], 
                                 lowBound = 0, cat = LpBinary)
expandKolkata

{'Kolkata': Expand_Kolkata}

In [10]:
# as Kolkata and Coimbatore have no choice but to remain open
openDepot['Kolkata'].varValue = 1
openDepot['Coimbatore'].varValue = 1

In [11]:
# Factory Output constraint
for src in plant:
    model.addConstraint(LpConstraint(e = lpSum(flowVar[src+'_'+dest] for dest in cost[src]),
                                     sense = LpConstraintLE,
                                     name = 'Factory_Capacity_From_' + src,
                                     rhs = supply[src]))

In [12]:
# Customer Demand constraint
for dest in distributor:
    model.addConstraint(LpConstraint(
        e = lpSum(flowVar[possibleFlows[i][0] + '_' + possibleFlows[i][1]] for i in range(0, len(possibleFlows)) if possibleFlows[i][1] == dest),
        sense = LpConstraintEQ,
        name = 'Customer_Demand_At_' + dest,
        rhs = demand[dest]))

In [13]:
# Depot capacity constraint
for src in depot:
    if src == 'Kolkata':
        model += (lpSum(flowVar[src+'_'+dest] for dest in cost[src]) <= (throughput[src] * openDepot[src]) + 
                  (20000 * expandKolkata[src]), "Depot_Capacity_From_"+src)
    else:
        model += (lpSum(flowVar[src+'_'+dest] for dest in cost[src]) <= throughput[src] * openDepot[src], "Depot_Capacity_From_"+src)
model

Shipping_Cost_Optimization_2:
MINIMIZE
None
SUBJECT TO
Factory_Capacity_From_NewDelhi: Flow_NewDelhi_Ahmedabad + Flow_NewDelhi_C1
 + Flow_NewDelhi_C3 + Flow_NewDelhi_C4 + Flow_NewDelhi_C6
 + Flow_NewDelhi_Chennai + Flow_NewDelhi_Coimbatore + Flow_NewDelhi_Kolkata
 + Flow_NewDelhi_Nagpur + Flow_NewDelhi_Patna <= 150000

Factory_Capacity_From_Vishakhapatnam: Flow_Vishakhapatnam_C1
 + Flow_Vishakhapatnam_Chennai + Flow_Vishakhapatnam_Coimbatore
 + Flow_Vishakhapatnam_Kolkata + Flow_Vishakhapatnam_Nagpur
 + Flow_Vishakhapatnam_Patna <= 200000

Customer_Demand_At_C1: Flow_Kolkata_C1 + Flow_NewDelhi_C1 + Flow_Patna_C1
 + Flow_Vishakhapatnam_C1 = 50000

Customer_Demand_At_C2: Flow_Ahmedabad_C2 + Flow_Chennai_C2
 + Flow_Coimbatore_C2 + Flow_Kolkata_C2 + Flow_Patna_C2 = 10000

Customer_Demand_At_C3: Flow_Ahmedabad_C3 + Flow_Coimbatore_C3
 + Flow_Kolkata_C3 + Flow_Nagpur_C3 + Flow_NewDelhi_C3 + Flow_Patna_C3 = 40000

Customer_Demand_At_C4: Flow_Chennai_C4 + Flow_Kolkata_C4 + Flow_Nagpur_C4
 + Fl

In [14]:
# Depot Throughput constraint
for src in depot:
    model.addConstraint(LpConstraint(
        e = lpSum(flowVar[possibleFlows[i][0] + '_' + possibleFlows[i][1]] for i in range(0, len(possibleFlows)) if possibleFlows[i][1] == src) - 
        lpSum(flowVar[src + '_' + dest] for dest in cost[src]),
        sense = LpConstraintEQ,
        name = 'Depot_Throughput_At_' + src,
        rhs = 0))

In [15]:
# Depot count
model.addConstraint(LpConstraint(e = lpSum(openDepot.values()),
                                 sense = LpConstraintLE,
                                 name = 'Depot_Count',
                                 rhs = 4))
model

Shipping_Cost_Optimization_2:
MINIMIZE
None
SUBJECT TO
Factory_Capacity_From_NewDelhi: Flow_NewDelhi_Ahmedabad + Flow_NewDelhi_C1
 + Flow_NewDelhi_C3 + Flow_NewDelhi_C4 + Flow_NewDelhi_C6
 + Flow_NewDelhi_Chennai + Flow_NewDelhi_Coimbatore + Flow_NewDelhi_Kolkata
 + Flow_NewDelhi_Nagpur + Flow_NewDelhi_Patna <= 150000

Factory_Capacity_From_Vishakhapatnam: Flow_Vishakhapatnam_C1
 + Flow_Vishakhapatnam_Chennai + Flow_Vishakhapatnam_Coimbatore
 + Flow_Vishakhapatnam_Kolkata + Flow_Vishakhapatnam_Nagpur
 + Flow_Vishakhapatnam_Patna <= 200000

Customer_Demand_At_C1: Flow_Kolkata_C1 + Flow_NewDelhi_C1 + Flow_Patna_C1
 + Flow_Vishakhapatnam_C1 = 50000

Customer_Demand_At_C2: Flow_Ahmedabad_C2 + Flow_Chennai_C2
 + Flow_Coimbatore_C2 + Flow_Kolkata_C2 + Flow_Patna_C2 = 10000

Customer_Demand_At_C3: Flow_Ahmedabad_C3 + Flow_Coimbatore_C3
 + Flow_Kolkata_C3 + Flow_Nagpur_C3 + Flow_NewDelhi_C3 + Flow_Patna_C3 = 40000

Customer_Demand_At_C4: Flow_Chennai_C4 + Flow_Kolkata_C4 + Flow_Nagpur_C4
 + Fl

In [16]:
# Model Objective
flowVariables = lpSum(lpSum(cost[src][dest] * flowVar[src + '_' + dest] for dest in cost[src]) for src in cost.keys())
costOpenDepots = lpSum(openDepot[d] * opencost[d] for d in depot)
objective = flowVariables + costOpenDepots + 3000 * expandKolkata['Kolkata']
objective
model.setObjective(objective)
model

Shipping_Cost_Optimization_2:
MINIMIZE
3000*Expand_Kolkata + 1.5*Flow_Ahmedabad_C2 + 0.5*Flow_Ahmedabad_C3 + 1.5*Flow_Ahmedabad_C5 + 1.0*Flow_Ahmedabad_C6 + 0.4*Flow_Chennai_C2 + 0.5*Flow_Chennai_C4 + 0.6*Flow_Chennai_C5 + 0.9*Flow_Chennai_C6 + 1.5*Flow_Coimbatore_C2 + 2.0*Flow_Coimbatore_C3 + 0.5*Flow_Coimbatore_C5 + 1.5*Flow_Coimbatore_C6 + 1.0*Flow_Kolkata_C1 + 0.5*Flow_Kolkata_C2 + 0.5*Flow_Kolkata_C3 + 1.0*Flow_Kolkata_C4 + 0.5*Flow_Kolkata_C5 + 0.2*Flow_Nagpur_C3 + 1.5*Flow_Nagpur_C4 + 0.5*Flow_Nagpur_C5 + 1.5*Flow_Nagpur_C6 + 0.5*Flow_NewDelhi_Ahmedabad + 1.0*Flow_NewDelhi_C1 + 1.5*Flow_NewDelhi_C3 + 2.0*Flow_NewDelhi_C4 + 1.0*Flow_NewDelhi_C6 + 0.4*Flow_NewDelhi_Chennai + 1.0*Flow_NewDelhi_Coimbatore + 0.5*Flow_NewDelhi_Kolkata + 0.2*Flow_NewDelhi_Nagpur + 0.6*Flow_NewDelhi_Patna + 1.2*Flow_Patna_C1 + 0.6*Flow_Patna_C2 + 0.5*Flow_Patna_C3 + 0.3*Flow_Patna_C5 + 0.8*Flow_Patna_C6 + 2.0*Flow_Vishakhapatnam_C1 + 0.3*Flow_Vishakhapatnam_Chennai + 0.5*Flow_Vishakhapatnam_Coimbatore +

In [17]:
model.solve()
print("Model Status = ", LpStatus[model.status])

Model Status =  Optimal


In [18]:
# Each of the variables is printed with it's resolved optimum value
df = pd.DataFrame()
variableName = []
variableValue = []
for v in model.variables():
#     print(v.name, "=", v.varValue)
    variableName.append(v.name)
    variableValue.append(v.varValue)

df['Variables'] = variableName
df['Variable_Values'] = variableValue
# The optimised objective function value is printed to the screen    
print("Product demand from all of our customers can be satisfied for a total cost of ", value(model.objective))

Product demand from all of our customers can be satisfied for a total cost of  189000.0


In [19]:
print("The optimal plan is as follows:")
df[df.Variable_Values > 0].reset_index(drop = True)

The optimal plan is as follows:


,Variables,Variable_Values
0,Expand_Kolkata,1.0
1,Flow_Chennai_C4,25000.0
2,Flow_Coimbatore_C5,10000.0
3,Flow_Kolkata_C2,10000.0
4,Flow_Kolkata_C4,10000.0
5,Flow_Kolkata_C5,50000.0
6,Flow_Nagpur_C3,40000.0
7,Flow_NewDelhi_C1,50000.0
8,Flow_NewDelhi_C6,20000.0
9,Flow_NewDelhi_Nagpur,40000.0


In [20]:
model.writeLP('ShippingCost2.lp')

[Expand_Kolkata,
 Flow_Ahmedabad_C2,
 Flow_Ahmedabad_C3,
 Flow_Ahmedabad_C5,
 Flow_Ahmedabad_C6,
 Flow_Chennai_C2,
 Flow_Chennai_C4,
 Flow_Chennai_C5,
 Flow_Chennai_C6,
 Flow_Coimbatore_C2,
 Flow_Coimbatore_C3,
 Flow_Coimbatore_C5,
 Flow_Coimbatore_C6,
 Flow_Kolkata_C1,
 Flow_Kolkata_C2,
 Flow_Kolkata_C3,
 Flow_Kolkata_C4,
 Flow_Kolkata_C5,
 Flow_Nagpur_C3,
 Flow_Nagpur_C4,
 Flow_Nagpur_C5,
 Flow_Nagpur_C6,
 Flow_NewDelhi_Ahmedabad,
 Flow_NewDelhi_C1,
 Flow_NewDelhi_C3,
 Flow_NewDelhi_C4,
 Flow_NewDelhi_C6,
 Flow_NewDelhi_Chennai,
 Flow_NewDelhi_Coimbatore,
 Flow_NewDelhi_Kolkata,
 Flow_NewDelhi_Nagpur,
 Flow_NewDelhi_Patna,
 Flow_Patna_C1,
 Flow_Patna_C2,
 Flow_Patna_C3,
 Flow_Patna_C5,
 Flow_Patna_C6,
 Flow_Vishakhapatnam_C1,
 Flow_Vishakhapatnam_Chennai,
 Flow_Vishakhapatnam_Coimbatore,
 Flow_Vishakhapatnam_Kolkata,
 Flow_Vishakhapatnam_Nagpur,
 Flow_Vishakhapatnam_Patna,
 Open_Ahmedabad,
 Open_Chennai,
 Open_Coimbatore,
 Open_Kolkata,
 Open_Nagpur,
 Open_Patna]